In [ ]:
from datetime import datetime
#vytvoření časové značky
timestamp = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
#vytvoření univerzální značky
import uuid 
id = uuid.uuid1()
import psycopg2
from rich import print

# Přihlašovací údaje k databázi
dbname = 'DP_pokus'
user = 'postgres'
password = 'kapli4ky'
host = 'localhost'
port = '5432'  # Výchozí port PostgreSQL

try:
    # Vytvoření připojení
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    # Vytvoření kurzoru pro provádění SQL dotazů
    cursor = connection.cursor()
    print("Připojení k databázi bylo úspěšné")

     # SQL dotaz na vybraní měřítek a roků z tabulky
    sql_query = """
    SELECT
     zkratka_txt, datovy_zdroj,
     indikator_cz, meritko, last_update, source_dat
    FROM
        metadata_dat
    """
    cursor.execute(sql_query)
    data = cursor.fetchall()

    # Vytvoření seznamů měřítek a roků
    meritka = [row[0] for row in data]
    roky = [row[1] for row in data]


    # Funkce pro určení barvy a poznámky na základě klasického měřítka
    def urci_barvu_klasicke_meritko(meritko):
        meritko_cislo = int(meritko.replace("1:", "").replace(" ", ""))
        if meritko_cislo <= 10000:
            return "#006400", ""  # tm. zelená, bez poznámky
        elif meritko_cislo <= 50000:
            return "#FFA500", "Měřítko má méně přesné rozlišení."  # oranžová, poznámka
        else:
            return "#FF0000", "Měřítko velmi hrubé - Data je potřeba interpretovat obezřetně."  # červená, poznámka

    # Funkce pro určení barvy a poznámky na základě měřítka v metrech na pixel
    def urci_barvu_meritko_na_pixel(meritko):
        meritko_cislo = int(meritko.replace("m/px", "").replace(" ", ""))
        if meritko_cislo <= 20:
            return "#006400", ""  # tm. zelená, bez poznámky
        elif meritko_cislo <= 100:
            return "#FFA500", "Měřítko má méně přesné rozlišení."  # oranžová, poznámka
        else:
            return "#FF0000", "Měřítko velmi hrubé - Data je potřeba interpretovat obezřetně."  # červená, poznámka
        
    # Funkce pro určení barvy na základě stáří roku
    def urci_barvu_rok(rok):
        aktualni_rok = datetime.now().year
        rozdil = aktualni_rok - rok
        if rozdil < 3:
            return "#1a9641"  # zelená
        elif rozdil < 5:
            return "#fdae61"  # oranžová
        else:
            return "#d7191c"  # červená
                
    # Generování HTML obsahu
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Zranitelnost krajiny</title>
        <meta charset='UTF-8'>
        <meta name='description' content='HTML stránka georeportu zranitelnosti krajiny'>
        <meta name='keywords' content='zranitelnost krajiny, georeport, ekologie'>
        <meta name='author' content='Adéla Petřeková'>
        <style>
            @page {{
                size: A4;
                margin: 10mm 20mm;
                @bottom-center {{
                    content: "Strana " counter(page);
                    font-family: Arial, sans-serif;
                    text-align: center; /* Uložení číslování strany na střed */
                }}
            }}
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
            }}
            .main
            h1 {{
                background-color: #4CAF50;
                color: #fff;
                padding: 10px;
            }}
            p {{
                padding: 10px;
            }}
            .footer {{
                position: fixed;
                bottom: 0;
                width: 100%;
                text-align: center;
            }}
            .footer hr {{
                width: 100%;
                border: none;
                border-top: 1px solid #000;
                margin: 0;
            }}
            .footer-content {{
                display: flex;
                justify-content: space-between;
                align-items: center;
                padding: 0 0;
            }}
            .footer img {{
                height: 50px;
            }}
            .page_number {{
                font-family: Arial, sans-serif;
            }}
            .indicator-box {{
                border: 1px solid #000;
                padding: 10px;
                margin: 10px 0;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
            }}
            th, td {{
                border: 1px solid #000;
                padding: 8px;
                text-align: left;
            }}
            th {{
                background-color: #f2f2f2;
            }}
            
        </style>
    </head>
    <body>
    <header>
        <h1>Název reportu: Zranitelnost krajiny</h1>
        <p>Lokalita: XXX (GPS souřadnice) <br>
        Souřadnice: MBR extentu <br>
        Datum a čas generování: {timestamp} <br>
        Unikatní identifikátor: {id}
        </p>
        </header>
        <main>
        <p> Tento report podává zprávu o degradaci krajiny na základě metody ESAI ....</p>

        <h2> Krok I: Náchylnost krajiny k degradaci </h2>
            <p> Ve zvoleném území dosahuje výsledná hodnota náchylnosti k degradaci hodnoty "XX" a je klasifikována jako "textový název kategorie (číslo kategorie)"</p>
            <div class="indicator-box">
                <!-- do ráměčku vložení 3 nejhorších indikátorů -->
                Nejhorší 3 indikátory v zájmovém území jsou XXX, YYY, ZZZ. (řazeno od nejhoršího)

            </div>
            <!-- vložení sloupcového grafu -->
            <h3> Věrohodnost dat </h3>
                <table>
                    <tr>
                        <th> Zkratka </th>
                        <th> Název </th>
                        <th> Datový zdroj </th>
                        <th> Garant </th>
                        <th> Měřítko </th>
                        <th> Rok aktualizace </th>
                    </tr>
    """

    # Přidání měřítek do tabulky
    for zkratka_txt, datovy_zdroj, indikator_cz, meritko, last_update, source_dat in data:
        if "1:" in meritko:
            barva, poznamka = urci_barvu_klasicke_meritko(meritko)
        elif "m/px" in meritko:
            barva, poznamka = urci_barvu_meritko_na_pixel(meritko)
        
        # Převedení last_update na datetime objekt
        barva_rok = urci_barvu_rok(last_update)

        html += f"""
                    <tr>
                        <td> {zkratka_txt} </td>
                        <td> {indikator_cz} </td>
                        <td> {datovy_zdroj} </td>
                        <td> {source_dat}  </td>
                        <td style="color: {barva};"> {meritko} {poznamka} </td>
                        <td style="color: {barva_rok};"> {last_update} </td>
                    </tr>
        """

    html += """
                </table>
                <!-- mapa území s celkovou kategorií -->

        <h2> Krok II: Posouzení funkčnosti krajiny </h2>
            <p> slouží k nalezení krajinných segmentů jejichž degradace bude mít velký/malý význam dopad na funkčnost krajiny </p>
            <table>
                <tr>
                    <th> </th>
                    <th> Hodnota </th>
                    <th> Kategorie </th>
                </tr>
                <tr>
                    <td> Zásoba uhlíku ve vegetaci </td>
                    <td> Hodnota 1 </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Evapotranspirace </td>
                    <td> Hodnota 1 </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Ekologická hodnota biotopu </td>
                    <td> Hodnota 1 </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Vodoretence </td>
                    <td> Hodnota 1 </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Integrovaná hodnota </td>
                    <td> --- </td>
                    <td> Kategorie 1 </td>
                </tr>
            </table>
            <!-- vložení map - 5x  -->
                <h3> Věrohodnost dat </h3>
                <table>
                    <tr>
                        <th> Název </th>
                        <th> Datový zdroj geometrie/KB</th>
                        <th> Garant </th>
                        <th> Měřítko </th>
                        <th> Rok aktualizace </th>
                    </tr>
   
                # Přidání měřítek do tabulky
   # for meritko in meritka:
    #    if "1:" in meritko:
     #       barva, poznamka = urci_barvu_klasicke_meritko(meritko)
      #  elif "m/px" in meritko:
       #     barva, poznamka = urci_barvu_meritko_na_pixel(meritko)
        #barva_rok = urci_barvu_rok(rok)

        
                    <tr>
                        <td> Indikátor 1 </td>
                        <td> Hodnota 1 </td>
                        <td> Věrohodnost 1 </td>
                        <td>  meritko</td>
                        <td>ROK </td>
                    </tr>
        """

    html += """
                </table>

        <h2> Krok III: Odolnost krajiny </h2>
            <!-- vložení mapy odolnosti -->

        <h2> Krok IV: Syntéze/lokalita prioritních opatření </h2>
            <!-- vložení mapy kde prioritně umístit opatření -->    

        </main>
        <footer>
            <div class="footer">
                <hr>
                <div class="footer-content">
                    <img src="katedra_logo.png" alt="Logo KGI">
                    <img src="logo_czechglobe.png" alt="Logo CzechGlobe">
                </div>
             </div>
        </footer>    
    </body>
    </html>
    """
 # Zde můžete uložit HTML do souboru nebo jej jinak zpracovat
    with open("report.html", "w", encoding="utf-8") as file:
        file.write(html)

    print("HTML report byl úspěšně vygenerován.")

except Exception as error:
    print(f"Chyba při připojování k databázi: {error}")

finally:
    if connection:
        cursor.close()
        connection.close()

Připojení k databázi bylo úspěšné

HTML report byl úspěšně vygenerován.

In [217]:
#vytvoření HTML souboru ze šablony
with open("index.html", "w") as f:
    f.write(html.format(timestamp=timestamp, id=id))
#otevření html souboru
#print("Generated index.html")
#import webbrowser
#webbrowser.open("index.html")

In [218]:
# output html as a PDF
# first, import the necessary modules

from weasyprint import HTML
# then, convert the HTML to a PDF
HTML("index.html").write_pdf("index.pdf")

print("Generated index.pdf")
#otevření pdf souboru
import webbrowser
webbrowser.open("index.pdf")

Generated index.pdf

True

In [20]:
from datetime import datetime
#vytvoření časové značky
timestamp = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
#vytvoření univerzální značky
import uuid 
id = uuid.uuid1()
import psycopg2
from rich import print

# Přihlašovací údaje k databázi
dbname = 'DP_pokus'
user = 'postgres'
password = 'kapli4ky'
host = 'localhost'
port = '5432'  # Výchozí port PostgreSQL

try:
    # Vytvoření připojení
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    # Vytvoření kurzoru pro provádění SQL dotazů
    cursor = connection.cursor()
    print("Připojení k databázi bylo úspěšné")

     # SQL dotaz na vybraní měřítek a roků z tabulky
    sql_query = """
    SELECT
     zkratka, indikator_cz, meritko, last_update, source_dat
    FROM
        metadata_dat
    """
    cursor.execute(sql_query)
    data = cursor.fetchall()

    # Vytvoření seznamů měřítek a roků
    meritka = [row[0] for row in data]
    roky = [row[1] for row in data]


    # Funkce pro určení barvy a poznámky na základě klasického měřítka
    def urci_barvu_klasicke_meritko(meritko):
        meritko_cislo = int(meritko.replace("1:", "").replace(" ", ""))
        if meritko_cislo <= 10000:
            return "#006400", ""  # tm. zelená, bez poznámky
        elif meritko_cislo <= 50000:
            return "#FFA500", "Měřítko má méně přesné rozlišení."  # oranžová, poznámka
        else:
            return "#FF0000", "Měřítko velmi hrubé - Data je potřeba interpretovat obezřetně."  # červená, poznámka

    # Funkce pro určení barvy a poznámky na základě měřítka v metrech na pixel
    def urci_barvu_meritko_na_pixel(meritko):
        meritko_cislo = int(meritko.replace("m/px", "").replace(" ", ""))
        if meritko_cislo <= 20:
            return "#006400", ""  # tm. zelená, bez poznámky
        elif meritko_cislo <= 100:
            return "#FFA500", "Měřítko má méně přesné rozlišení."  # oranžová, poznámka
        else:
            return "#FF0000", "Měřítko velmi hrubé - Data je potřeba interpretovat obezřetně."  # červená, poznámka
        
    # Funkce pro určení barvy na základě stáří roku
    def urci_barvu_rok(rok):
        aktualni_rok = datetime.now().year
        rozdil = aktualni_rok - rok
        if rozdil < 3:
            return "#1a9641"  # zelená
        elif rozdil < 5:
            return "#fdae61"  # oranžová
        else:
            return "#d7191c"  # červená

    # Generování HTML obsahu
    html = """
    <!DOCTYPE html>
    <html lang="cs">
    <head>
        <meta charset="UTF-8">
        <title>Georeport Formulář</title>
        <script src="https://js.arcgis.com/4.15/"></script>
        <link rel="stylesheet" href="https://js.arcgis.com/4.15/esri/themes/light/main.css">
        <style>
            body {
                display: flex;
            }
            #formColumn {
                width: 20%;
                padding: 20px;
            }
            #mapColumn {
                width: 80%;
                padding: 20px;
            }
            #map {
                width: 100%;
                height: 600px; /* Nastavte výšku mapového pole podle potřeby */
                background-color: #eaeaea; /* Dočasná barva pozadí pro mapové pole */
            }
        </style>
    </head>
    <body>
        <div id="formColumn">
            <h1>Georeport</h1>
            <form id="georeportForm" action="process_form.php" method="post">
                <!-- Výběr území -->
                <h2>Výběr území</h2>
                <label>
                    <input type="radio" name="uzemi" value="cele" checked>
                    Celé území
                </label>
                <br>
                <label>
                    <input type="radio" name="uzemi" value="uzivatelske">
                    Uživatelské území
                </label>
                <div id="spatialSelection" style="display: none;">
                    <!-- Nástroj pro výběr území -->
                    <p>Vyberte území pomocí kvadrátů (q):</p>
                    <!-- Zde by měl být nástroj pro výběr území -->
                </div>

                <!-- Výběr tématu -->
                <h2>Výběr tématu</h2>
                    <label>
                        <input type="checkbox" name="tema[]" value="nachylnost" checked>
                        Náchylnost k degradaci mapa
                    </label>
                    <br>
                    <label>
                        <input type="checkbox" name="tema[]" value="funkcnost" checked>
                        Funkčnost krajiny
                    </label>
                    <br>
                    <label>
                        <input type="checkbox" name="tema[]" value="odolnost" checked>
                        Odolnost krajiny
                    </label>
                    <br>
                    <label>
                        <input type="checkbox" name="tema[]" value="synteza" checked>
                        Syntéza/lokalizace prioritních opatření
                    </label>
                    <br>
                    <label>
                        <input type="checkbox" name="tema[]" value="typizace" checked>
                        Typizace prioritních opatření
                    </label>

                <!-- Odeslání formuláře -->
                <br><br>
                <button type="submit">Odeslat</button>
            </form>
        </div>
        <div id="mapColumn">
            <div id="map"></div>
        </div>

        <script>
            require([
                "esri/config",
                "esri/portal/Portal",
                "esri/portal/PortalItem",
                "esri/views/MapView",
                "esri/WebMap",
                "esri/widgets/Sketch",
                "esri/layers/GraphicsLayer",
                "esri/widgets/LayerList",
                "esri/widgets/BasemapToggle",
                "esri/Basemap",
                "esri/layers/GroupLayer"
            ], function(esriConfig, Portal, PortalItem, MapView, WebMap, Sketch, GraphicsLayer, LayerList, BasemapToggle, Basemap, GroupLayer) {
                esriConfig.portalUrl = "https://www.arcgis.com"; // Ujistěte se, že je správně nakonfigurován portál

                var portal = new Portal();
                portal.load().then(function() {
                    var portalItem = new PortalItem({
                        id: "01f4cb6275ae44b0847a8d6814bab63b" // Zde vložte ID vaší Web Mapping Application
                    });

                    portalItem.load().then(function() {
                        var webmap = new WebMap({
                            portalItem: portalItem
                        });

                        var view = new MapView({
                            container: "map",
                            map: webmap,
                            center: [14.96, 49.37], // Souřadnice středu mapy
                            zoom: 11,
                            spatialReference: {
                                wkid: 3857 // Nastavte prostorovou referenci na EPSG:3857
                            }
                        });

                        var graphicsLayer = new GraphicsLayer();
                        webmap.add(graphicsLayer);

                    
                        // Přidání přepínače podkladových map
                        var basemapToggle = new BasemapToggle({
                            view: view,
                            nextBasemap: new Basemap({
                                portalItem: {
                                    id: "86265e5a4bbb4187a59719cf134e0018" // Zde vložte ID hybridní podkladové mapy
                                }
                            })
                        });
                        view.ui.add(basemapToggle, "bottom-right");
                    });
                });
            });

            document.querySelector('input[name="uzemi"][value="uzivatelske"]').addEventListener('change', function() {
                document.getElementById('spatialSelection').style.display = 'block';
            });
            document.querySelector('input[name="uzemi"][value="cele"]').addEventListener('change', function() {
                document.getElementById('spatialSelection').style.display = 'none';
            });

            // JavaScript pro zobrazení obsahu podle zaškrtnutých políček
            document.getElementById('georeportForm').addEventListener('submit', function(event) {
                event.preventDefault(); // Zabrání odeslání formuláře

                // Skryje všechny sekce
                document.getElementById('nachylnost').style.display = 'none';
                document.getElementById('funkcnost').style.display = 'none';
                document.getElementById('odolnost').style.display = 'none';
                document.getElementById('synteza').style.display = 'none';

                // Zobrazí sekce podle zaškrtnutých políček
                var checkboxes = document.querySelectorAll('input[name="tema[]"]:checked');
                checkboxes.forEach(function(checkbox) {
                    var sectionId = checkbox.value;
                    document.getElementById(sectionId).style.display = 'block';
                });
            });
        </script>
    </body>
    </html>
    """
# Zde můžete uložit HTML do souboru nebo jej jinak zpracovat
    with open("report.html", "w", encoding="utf-8") as file:
        file.write(html)

    print("HTML report byl úspěšně vygenerován.")

except Exception as error:
    print(f"Chyba při připojování k databázi: {error}")

finally:
    if connection:
        cursor.close()
        connection.close()

Připojení k databázi bylo úspěšné

HTML report byl úspěšně vygenerován.